# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Import Required Libraries 📚</h1></span>

In [1]:
import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
from torcheval.metrics.functional import binary_auroc

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold 

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

/root/anaconda3/envs/isic/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Training Configuration ⚙️</h1></span>

In [2]:
CONFIG = {
    "seed": 42,
    "epochs": 50,
    "img_size": 224,
    "model_name": "tf_efficientnetv2_s", # efficientnet-b0
    "train_batch_size": 32,
    "valid_batch_size": 32, 
    "learning_rate": 1e-4,
    "scheduler": 'OneCycleLR',
    "min_lr": 1e-6,
    "weight_decay": 1e-6,
    "fold" : 0,
    "n_fold": 5,
    "n_accumulate": 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "early_stopping_patience": 5,  # Early Stopping의 patience 설정
    "early_stopping_delta": 0.001,  # Early Stopping의 개선 기준 설정
    'max_lr': 1e-3,
    'epochs': 50,
    'steps_per_epoch': 100,
    'pct_start': 0.3,  # 워밍업 비율
    'anneal_strategy': 'cos',  # 학습률 감소 전략
    'final_div_factor': 1e4,  # 마지막 학습률 감소 비율
    'div_factor': 25.0,  # 초기 학습률 감소 비율
}

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Set Seed for Reproducibility</h1></span>

In [3]:
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(CONFIG['seed'])

In [4]:
def get_train_file_path(image_id):
    return f"train-image/image/{image_id}.jpg"

# <h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Read the Data 📖</h1>

In [5]:
train_images = sorted(glob.glob("train-image/image/*.jpg"))

In [6]:
df = pd.read_csv("data/train-metadata.csv")

df['target'].value_counts()

target
0    400666
1       393
Name: count, dtype: int64

In [7]:
from sklearn.utils import resample

df_positive = df[df["target"] == 1].reset_index(drop=True)
df_negative = df[df["target"] == 0].reset_index(drop=True)

df_negative_downsample = resample(df_negative, 
                                   replace=False,    # 복원 샘플링하지 않음
                                   n_samples=len(df_positive) * 3,  # 소수 클래스의 10배 크기로 샘플링 (임의 값)
                                   random_state=42)  # 재현성을 위한 랜덤 시드 설정

df = pd.concat([df_negative_downsample, df_positive])

df['file_path'] = df['isic_id'].apply(get_train_file_path)
df = df[ df["file_path"].isin(train_images) ].reset_index(drop=True)
df['target'].value_counts()

target
0    1179
1     393
Name: count, dtype: int64

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Create Folds</h1></span>

In [8]:
sgkf = StratifiedGroupKFold(n_splits=CONFIG['n_fold'])

for fold, ( _, val_) in enumerate(sgkf.split(df, df.target,df.patient_id)):
      df.loc[val_ , "kfold"] = int(fold)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Dataset Class</h1></span>

In [9]:
import random
import cv2
from torch.utils.data import Dataset
from albumentations.pytorch import ToTensorV2

class ISICDataset_for_Train(Dataset):
    def __init__(self, df, transforms=None):
        """
        데이터프레임을 기반으로 클래스 비율을 1:1로 맞춘 데이터셋 생성.
        소수 클래스(positive)에 대해서만 데이터 증강을 수행하여 다수 클래스(negative)와 비율을 맞춤.
        """
        self.transforms = transforms
        
        # 소수 클래스(positive)와 다수 클래스(negative) 분리
        self.df_positive = df[df["target"] == 1].reset_index(drop=True)
        self.df_negative = df[df["target"] == 0].reset_index(drop=True)
        
        # 다수 클래스의 수에 맞추기 위해 소수 클래스 데이터 증강
        self.file_names_positive = self.df_positive['file_path'].values
        self.file_names_negative = self.df_negative['file_path'].values
        self.targets_positive = self.df_positive['target'].values
        self.targets_negative = self.df_negative['target'].values
        
        # 소수 클래스(positive)를 다수 클래스(negative)와 동일한 수로 증강
        self.num_to_augment = len(self.df_negative) - len(self.df_positive)
        if self.num_to_augment > 0:
            augmented_positive_samples = random.choices(list(zip(self.file_names_positive, self.targets_positive)), k=self.num_to_augment)
            self.file_names_positive = np.concatenate([self.file_names_positive, [x[0] for x in augmented_positive_samples]])
            self.targets_positive = np.concatenate([self.targets_positive, [x[1] for x in augmented_positive_samples]])
        
        # 결합하여 최종 데이터셋 구성
        self.file_names = np.concatenate([self.file_names_negative, self.file_names_positive])
        self.targets = np.concatenate([self.targets_negative, self.targets_positive])

    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        target = self.targets[index]
        
        # 데이터 증강 수행
        if self.transforms:
            img = self.transforms(image=img)["image"]
        else:
            img = ToTensorV2()(image=img)["image"]
            
        return {
            'image': img,
            'target': target
        }

In [10]:
class ISICDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.targets = df['target'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = self.load_image(img_path)
        target = self.targets[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': img,
            'target': target
        }
    
    def load_image(self, path):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Augmentations</h1></span>

In [11]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=90, p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit=45, p=0.5),
        A.CoarseDropout(max_holes=8, max_height=16, max_width=16, min_holes=1, min_height=8, min_width=8, fill_value=0, p=0.5),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
        A.GaussianBlur(blur_limit=(3, 7), p=0.3),
        A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ], p=1.0),
    
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ], p=1.0)
}

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">GeM Pooling</h1></span>

In [12]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6, p_clamp_min=1.0, p_clamp_max=6.0, dynamic_p=False):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1) * p)  # 학습 가능한 p 파라미터 초기화
        self.eps = eps  # 작은 값으로 인해 0으로 나누는 것을 방지
        self.p_clamp_min = p_clamp_min
        self.p_clamp_max = p_clamp_max
        self.dynamic_p = dynamic_p
        
        if self.dynamic_p:
            # 동적으로 p 값을 계산하는 레이어 추가
            self.p_layer = nn.Sequential(
                nn.Conv2d(1, 1, kernel_size=1),
                nn.Sigmoid()
            )
    
    def forward(self, x):
        if self.dynamic_p:
            # p 값을 입력에 따라 동적으로 계산
            p = self.p_layer(x.mean(dim=(2, 3), keepdim=True))
            p = p * (self.p_clamp_max - self.p_clamp_min) + self.p_clamp_min  # p 값 범위 조정
        else:
            # 정적 p 값 사용
            p = self.p.clamp(min=self.p_clamp_min, max=self.p_clamp_max)
        
        return self.gem(x, p, self.eps)
        
    def gem(self, x, p, eps):
        # GeM 수식 적용
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        # 클래스 설명 문자열 반환
        return (f"{self.__class__.__name__}(p={self.p.data.item():.4f}, eps={self.eps}, "
                f"p_clamp_min={self.p_clamp_min}, p_clamp_max={self.p_clamp_max}, "
                f"dynamic_p={self.dynamic_p})")



# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Create Model</h1></span>

In [13]:
class ISICModel(nn.Module):
    def __init__(self, model_name, num_classes=1, pretrained = True):
        super(ISICModel, self).__init__()
        # 모델 생성 및 사전 학습된 가중치 로드
        self.model = timm.create_model(model_name, pretrained=pretrained)

        # EfficientNet의 마지막 레이어 조정
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        
        # GeM Pooling 및 새로운 분류 레이어 추가
        self.pooling = GeM()
        self.dropout = nn.Dropout(p=0.6)  # Dropout 추가
        self.linear = nn.Linear(in_features, num_classes)

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        pooled_features = self.dropout(pooled_features)  # Dropout 적용
        output = self.linear(pooled_features)
        return output


model = ISICModel(model_name=CONFIG['model_name'])
model.to(CONFIG['device'])


INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/tf_efficientnetv2_s.in21k_ft_in1k)
INFO:timm.models._hub:[timm/tf_efficientnetv2_s.in21k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


ISICModel(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (drop_path): Identity()
        )
        (1): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Loss Function</h1></span>

In [203]:
# Focal Loss 정의
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, class_weights=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha  # 추가적인 클래스 가중치 (optional)
        self.class_weights = class_weights  # 클래스별 가중치
        self.reduction = reduction  # 'mean', 'sum', 'none'

    def forward(self, inputs, targets):
        # BCEWithLogitsLoss를 사용하여 기본 BCE 손실을 계산
        bce_loss = F.binary_cross_entropy_with_logits(inputs, targets, weight=self.class_weights, reduction='none')
        
        # 모델 출력을 확률로 변환
        probs = torch.sigmoid(inputs)
        
        # (1 - p_t)^gamma로 가중치 부여
        p_t = probs * targets + (1 - probs) * (1 - targets)
        focal_loss = bce_loss * (1 - p_t) ** self.gamma
        
        # 추가적인 클래스 가중치 적용 (alpha)
        if self.alpha is not None:
            alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
            focal_loss = alpha_t * focal_loss

        # reduction 방식에 따라 결과 반환
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# 데이터 불균형을 고려한 클래스 가중치 계산
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df['target']),
    y=df['target'].values
)

class_weights = torch.tensor([class_weights[1]], dtype=torch.float).to(CONFIG['device'])

# Focal Loss 사용 (클래스 가중치 포함)
criterion = FocalLoss(gamma=2.0, class_weights=class_weights)

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Training Function</h1></span>

In [204]:
from sklearn.metrics import roc_auc_score
from torcheval.metrics.functional import binary_auroc

def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        outputs = model(images).squeeze()
        targets = targets.view(-1) 
    
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        
        loss.backward()
        
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()
            optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
                
        probs = torch.sigmoid(outputs) # 추가
                
        auroc = binary_auroc(input=probs, target=targets).item() # outputs.squeeze() -> probs 
        running_auroc  += (auroc * batch_size)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size if running_auroc > 0 else 0
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, Train_Auroc=epoch_auroc,LR=optimizer.param_groups[0]['lr'])
    
    gc.collect()
    
    return epoch_loss, epoch_auroc


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Validation Function</h1></span>

In [205]:
@torch.inference_mode()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    
    for step, data in bar:        
        images = data['image'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = images.size(0)

        outputs = model(images).squeeze()
        targets = targets.view(-1) 

        loss = criterion(outputs, targets)
        
        probs = torch.sigmoid(outputs) # 추가
        
        auroc = binary_auroc(input=probs, target=targets).item() # outputs.squeeze() -> probs
        
        running_loss += (loss.item() * batch_size)  
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_auroc = running_auroc / dataset_size
    
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, Valid_Auroc=epoch_auroc, LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss, epoch_auroc


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Run Training</h1></span>

In [206]:
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0.0005):  # patience와 min_delta를 조정
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True


In [207]:
import time
import copy
import torch
import numpy as np
import gc
import wandb


def run_training(model, optimizer, scheduler, device, num_epochs):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    # WandB 초기화
    wandb.init(project="ISIC_classification_project", config=CONFIG)
    wandb.watch(model, log="all")
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = -np.inf
    
    early_stopping = EarlyStopping(patience=CONFIG['early_stopping_patience'], min_delta=CONFIG['early_stopping_delta'])
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        
        train_epoch_loss, train_epoch_auroc = train_one_epoch(model, optimizer, scheduler, dataloader=train_loader, device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss, val_epoch_auroc = valid_one_epoch(model, dataloader=valid_loader, device=CONFIG['device'], epoch=epoch)
    
        # WandB에 로그 기록
        wandb.log({
            "Train Loss": train_epoch_loss,
            "Valid Loss": val_epoch_loss,
            "Train AUROC": train_epoch_auroc,
            "Valid AUROC": val_epoch_auroc,
            "Learning Rate": scheduler.get_last_lr()[0]
        })
        
        # 모델 성능이 개선되었을 때만 저장
        if best_epoch_auroc <= val_epoch_auroc:
            print(f"Validation AUROC Improved ({best_epoch_auroc:.4f} ---> {val_epoch_auroc:.4f})")
            best_epoch_auroc = val_epoch_auroc
            best_model_wts = copy.deepcopy(model.state_dict())
            r = 'model_efficientnetv2_2/'
            PATH = r + f"AUROC{val_epoch_auroc:.4f}_Loss{val_epoch_loss:.4f}_epoch{epoch}.bin"
            torch.save(model.state_dict(), PATH)
            print(f"Model Saved")
            
            
        # Early Stopping 체크
        early_stopping(val_epoch_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best AUROC: {:.4f}".format(best_epoch_auroc))
    
    # 가장 성능이 좋았던 모델 가중치 로드
    model.load_state_dict(best_model_wts)
    
    # WandB 실행 종료
    wandb.finish()
    
    return model


In [208]:
from torch.optim import lr_scheduler

def fetch_scheduler(optimizer):
    scheduler = lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=CONFIG['max_lr'],  # 최대 학습률
        epochs=CONFIG['epochs'],
        steps_per_epoch=CONFIG['steps_per_epoch'],
        pct_start=CONFIG.get('pct_start', 0.3),  # OneCycleLR의 warmup 시작 비율 (기본값 0.3)
        anneal_strategy=CONFIG.get('anneal_strategy', 'cos'),  # 'cos' 또는 'linear'
        final_div_factor=CONFIG.get('final_div_factor', 1e4),  # 마지막에 학습률을 얼마나 줄일지 결정
        div_factor=CONFIG.get('div_factor', 25.0)  # 초기 학습률 = max_lr/div_factor
    )
    return scheduler


In [209]:
def prepare_loaders(df, fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    train_dataset = ISICDataset_for_Train(df=df_train, transforms=data_transforms['train'])
    valid_dataset = ISICDataset(df = df_valid, transforms=data_transforms["valid"])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader


<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">Prepare Dataloaders</span>

In [210]:
train_loader, valid_loader = prepare_loaders(df, fold=CONFIG["fold"])

<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">Define Optimizer and Scheduler</span>

In [211]:
# 옵티마이저 설정
optimizer = optim.Adam(
    model.parameters(),
    lr=CONFIG['learning_rate'], 
    weight_decay=CONFIG['weight_decay']
)

# 스케줄러 설정
scheduler = fetch_scheduler(optimizer)

<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;">Start Training</span>

In [212]:
model = run_training(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    device=CONFIG['device'],
    num_epochs=CONFIG['epochs']
)

100%|██████████| 8/8 [00:38<00:00,  4.79s/it, Epoch=1, LR=4.16e-5, Valid_Auroc=0.499, Valid_Loss=0.268]


Validation AUROC Improved (-inf ---> 0.4992)
Model Saved



100%|██████████| 8/8 [00:37<00:00,  4.72s/it, Epoch=2, LR=4.64e-5, Valid_Auroc=0.513, Valid_Loss=0.25] 


Validation AUROC Improved (0.4992 ---> 0.5132)
Model Saved



100%|██████████| 8/8 [00:37<00:00,  4.65s/it, Epoch=3, LR=5.44e-5, Valid_Auroc=0.532, Valid_Loss=0.226]


...

100%|██████████| 8/8 [04:18<00:00, 32.30s/it, Epoch=19, LR=0.000511, Valid_Auroc=0.566, Valid_Loss=0.147]


100%|██████████| 8/8 [04:15<00:00, 31.94s/it, Epoch=20, LR=0.000551, Valid_Auroc=0.56, Valid_Loss=0.15]  


100%|██████████| 8/8 [04:42<00:00, 35.29s/it, Epoch=21, LR=0.00059, Valid_Auroc=0.565, Valid_Loss=0.157]


Early stopping triggered
Training complete in 19h 13m 57s
Best AUROC: 0.5678


Learning Rate,▁▁▁▁▁▂▂▂▃▃▃▄▄▅▅▆▆▇▇██
Train AUROC,▁▂▃▃▄▄▅▆▆▆▇▇▇▇▇██████
Train Loss,█▆▅▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Valid AUROC,▁▂▄▆▆▇▆▇▇▇▇███▇████▇█
Valid Loss,█▇▆▄▄▄▃▃▃▃▂▂▃▂▂▁▂▁▂▂▂
Learning Rate,0.00059
Train AUROC,0.94959
Train Loss,0.1227
Valid AUROC,0.56468
Valid Loss,0.15651
